# 8 Puzzle solver
* Parsa KamaliPour - 97149081
* In this repository we're going to solve this puzzle using $ A^* $ and $ IDA $

#### imports:

In [2]:
import copy

import pandas as pd
import numpy as np
import collections
import heapq

#### Test case 1:

In [3]:
input_puzzle_1 = [
    [1, 2, 3],
    [4, 0, 5],
    [7, 8, 6]
]

print('Input: ')
print(pd.DataFrame(input_puzzle_1))
print()

desired_output_1 = [
    [1, 2, 3],
    [4, 5, 0],
    [7, 8, 6]
]

print('Desired Output:')
print(pd.DataFrame(desired_output_1))


Input: 
   0  1  2
0  1  2  3
1  4  0  5
2  7  8  6

Desired Output:
   0  1  2
0  1  2  3
1  4  5  0
2  7  8  6


#### Test case 2: (Hardest)

In [4]:
input_puzzle_2 = [
    [8, 6, 7],
    [2, 5, 4],
    [3, 0, 1]
]

print('Input 2: ')
print(pd.DataFrame(input_puzzle_2))
print()

desired_output_2 = [
    [6, 4, 7],
    [8, 5, 0],
    [3, 2, 1]
]

print('Desired Output 2:')
print(pd.DataFrame(desired_output_2))


Input 2: 
   0  1  2
0  8  6  7
1  2  5  4
2  3  0  1

Desired Output 2:
   0  1  2
0  6  4  7
1  8  5  0
2  3  2  1


### code's configs:

In [15]:
heuristic_method = input("Enter the desired Heuristic method: h1 or h2")
f_function_omega = eval(input("Enter the desired f function omega: 2 is Greedy, "
                         "0 is Uninformed best-first search, "
                         "0 < omega <= 1 is A*"))

test_case = eval(input("which test case? 1:easy, 2:hard"))

if test_case == 1:
    input_puzzle = input_puzzle_1
    desired_output = desired_output_1
elif test_case == 2:
    input_puzzle = input_puzzle_2
    desired_output = desired_output_2

### Matrix to dictionary converter

In [6]:
class Mat2dict:

    def __init__(self, matrix):
        self.matrix = matrix
        self.dic = {}


    def convert(self):
        for r in range(len(self.matrix)):
            for c in range(len(self.matrix[0])):
                key = self.matrix[r][c]
                self.dic[key] = [r, c]

        return self.dic


### the heuristic calculator class:

* H1 heuristic (misplaced tiles)

    $ \Sigma_{i=1}^{9} \; if \; currentPuzzleBoard[node_i] \; != \; goalPuzzleBoard[node_i]$

    $ then \; h(state_y) = h(state_y) + 1$

* H2 heuristic (manhattan distance)

    $ goalPuzzleBoard.find(currentPuzzleBoard[node_i]) $

    $ retrieve \; Row \; \& \; Col \; of \; goal $

    $ manhattanDistance = |(goal.row - current.row)| + |(goal.col - current.col)| $

    $ TotalHeuristic[state_i] = \Sigma_{i=1}^{9} manhattanDistance_i $

In [7]:
class Heuristic:

    def __init__(self, node, current_puzzle, desired_answer, method):
        self.method = method
        self.node = node
        self.current_puzzle = current_puzzle
        self.desired_answer = desired_answer
        #self.current_puzzle_dict = Mat2dict(self.current_puzzle)
        self.desired_answer_dict = Mat2dict(self.desired_answer).convert()

    def do(self):
        if self.method == 'h1':
            return self.h1_misplaced_tiles()
        elif self.method == 'h2':
            return self.h2_manhattan_distance()

    def h1_misplaced_tiles(self):
        misplaced_counter = 0
        for row in range(len(self.current_puzzle)):
            for col in range(len(self.current_puzzle[0])):
                if self.current_puzzle[row][col] != self.desired_answer[row][col]:
                    misplaced_counter += 1
        return misplaced_counter

    def h2_manhattan_distance(self):
        total_distance_counter = 0
        for row in range(len(self.current_puzzle)):
            for col in range(len(self.current_puzzle[0])):
                key = self.current_puzzle[row][col]
                correct_row, correct_col = self.desired_answer_dict[key]
                total_distance_counter += abs(row - correct_row) + abs(col - correct_col)
        return total_distance_counter

### The node class:

* F function is calculated in a such way that you can control how the Heuristic and G-cost
can perform:

    $ FCost(n) = (2-\omega) * GCost(n) + \omega * h(n)$

    $ if \; \omega = 2: $

    $ then: algorithm \; is \; Greedy \; due \; to \; GCost \; being \; 0:$

    $ FCost(n) = 0 + 2 * h(n) $

    $ if \; \omega = 0: $

    $ then: algorithm \; is \; uninformed \; search \; due \; to \; h(n) \; being \; 0:$

    $ FCost(n) = 2 * GCost(n) + 0 $

    $ if \; 0 \lt \omega \le 1 : $

    $ then: algorithm \; is \; informed \; search(A^*):$

    $ FCost(n) = (2-\omega) * GCost(n) + \omega * h(n) $

In [8]:
class Node:

    def __init__(self, current_puzzle, parent=None):
        self.current_puzzle = current_puzzle
        self.parent = parent

        if self.parent:
            self.g_cost = self.parent.f_function
            self.depth = self.parent.depth + 1

        else:
            self.g_cost = 0
            self.depth = 0

        self.h_cost = Heuristic(self, current_puzzle, desired_output, heuristic_method).do()
        self.f_function = (2 - f_function_omega) * self.g_cost + f_function_omega * self.h_cost

    def __eq__(self, other):
        return self.f_function == other.f_function

    def __lt__(self, other):
        return self.f_function < other.f_function

    def get_id(self):
        return str(self)

    def get_path(self):
        node, path = self, []
        while node:
            path.append(node)
            node = node.parent
        return list(reversed(path))

    def get_position(self, element):
        for row in range(len(self.current_puzzle)):
            for col in range(len(self.current_puzzle[0])):
                if self.current_puzzle[row][col] == element:
                    return [row, col]
        return [0, 0]



### The puzzle solver class:

In [9]:

class PuzzleSolver:

    def __init__(self, start_node):

        self.final_state = None
        self.start_node = start_node
        self.depth = 0
        self.visited_nodes = set()
        self.expanded_nodes = 0

    def solve(self):

        queue = [self.start_node]
        self.visited_nodes.add(self.start_node.get_id())

        while queue:
            self.expanded_nodes += 1
            node = heapq.heappop(queue)
            if node.current_puzzle == desired_output:
                self.final_state = node
                Result(self.final_state, self.expanded_nodes)
                return True

            if node.depth + 1 > self.depth:
                self.depth = node.depth + 1

            for neighbor in NeighborsCalculator(node).get_list_of_neighbors():
                if not neighbor.get_id in self.visited_nodes:
                    self.visited_nodes.add(neighbor.get_id())
                    heapq.heappush(queue, neighbor)

        return False



### result class

In [10]:
class Result:

    def __init__(self, final_state, expanded_nodes):
        self.expanded_nodes = expanded_nodes
        self.final_state = final_state
        self.solved_puzzle = self.final_state.current_puzzle
        self.path = self.final_state.get_path()
        self.show_puzzles()
        self.show_path()

    def show_puzzles(self):
        print("Inital Puzzle: ")
        print(pd.DataFrame(input_puzzle))
        print("Result Puzzle: ")
        print(pd.DataFrame(self.solved_puzzle))
        print("Expected Puzzle: ")
        print(pd.DataFrame(desired_output))
        print()
        print("Number of expanded nodes: {}".format(self.expanded_nodes))
        print()

    def show_path(self):
        counter = 0
        while self.path:
            counter += 1
            step = self.path.pop(0)
            print("step {}: ".format(counter))
            print(pd.DataFrame(step.current_puzzle))


### Neighbors calculator

In [11]:
class NeighborsCalculator:

    def __init__(self, current_state):
        self.current_state = current_state
        self.puzzle = self.current_state.current_puzzle
        self.neighbors = []

    def get_list_of_neighbors(self):
        row, col = map(int, self.current_state.get_position(0))
        #if row or col is None:
          #  return []

        # move right
        if col < len(self.puzzle[0]) - 1:
            moved_right = copy.deepcopy(self.puzzle)
            moved_right[row][col], moved_right[row][col + 1] = moved_right[row][col + 1], moved_right[row][col]
            self.neighbors.append(Node(moved_right, self.current_state))

        # move left
        if col > 0:
            moved_left = copy.deepcopy(self.puzzle)
            moved_left[row][col], moved_left[row][col - 1] = moved_left[row][col - 1], moved_left[row][col]
            self.neighbors.append(Node(moved_left, self.current_state))

        # move up
        if row > 0:
            moved_up = copy.deepcopy(self.puzzle)
            moved_up[row][col], moved_up[row - 1][col] = moved_up[row - 1][col], moved_up[row][col]
            self.neighbors.append(Node(moved_up, self.current_state))


        # move down
        if row < len(self.puzzle) - 1:
            moved_down = copy.deepcopy(self.puzzle)
            moved_down[row][col], moved_down[row + 1][col] = moved_down[row + 1][col], moved_down[row][col]
            self.neighbors.append(Node(moved_down, self.current_state))

        return self.neighbors

In [12]:
initial_state = Node(input_puzzle)

PuzzleSolver(initial_state).solve()


Inital Puzzle: 
   0  1  2
0  1  2  3
1  4  0  5
2  7  8  6
Result Puzzle: 
   0  1  2
0  1  2  3
1  4  5  0
2  7  8  6
Expected Puzzle: 
   0  1  2
0  1  2  3
1  4  5  0
2  7  8  6

Number of expanded nodes: 2

step 1: 
   0  1  2
0  1  2  3
1  4  0  5
2  7  8  6
step 2: 
   0  1  2
0  1  2  3
1  4  5  0
2  7  8  6


True

### The puzzle solver IDA class:

In [13]:

class PuzzleSolverIDA:

    def __init__(self, start_node, iterate):

        self.iterate = iterate
        self.final_state = None
        self.start_node = start_node
        self.depth = 0
        self.visited_nodes = set()
        self.expanded_nodes = 0
        self.f_cutoff = 0

    def solve(self):

        while True:
            self.f_cutoff += self.iterate
            queue = [self.start_node]
            self.visited_nodes.add(self.start_node.get_id())

            while queue:
                self.expanded_nodes += 1
                node = heapq.heappop(queue)
                if node.current_puzzle == desired_output:
                    self.final_state = node
                    Result(self.final_state, self.expanded_nodes)
                    return True

                if node.depth + 1 > self.depth:
                    self.depth = node.depth + 1

                for neighbor in NeighborsCalculator(node).get_list_of_neighbors():
                    if not neighbor.get_id in self.visited_nodes:
                        if neighbor.f_function <= self.f_cutoff:
                            self.visited_nodes.add(neighbor.get_id())
                            heapq.heappush(queue, neighbor)





In [16]:
initial_state = Node(input_puzzle)

PuzzleSolverIDA(initial_state, 4).solve()







Inital Puzzle: 
   0  1  2
0  8  6  7
1  2  5  4
2  3  0  1
Result Puzzle: 
   0  1  2
0  6  4  7
1  8  5  0
2  3  2  1
Expected Puzzle: 
   0  1  2
0  6  4  7
1  8  5  0
2  3  2  1

Number of expanded nodes: 2191847

step 1: 
   0  1  2
0  8  6  7
1  2  5  4
2  3  0  1
step 2: 
   0  1  2
0  8  6  7
1  2  5  4
2  3  1  0
step 3: 
   0  1  2
0  8  6  7
1  2  5  0
2  3  1  4
step 4: 
   0  1  2
0  8  6  7
1  2  0  5
2  3  1  4
step 5: 
   0  1  2
0  8  6  7
1  0  2  5
2  3  1  4
step 6: 
   0  1  2
0  0  6  7
1  8  2  5
2  3  1  4
step 7: 
   0  1  2
0  6  0  7
1  8  2  5
2  3  1  4
step 8: 
   0  1  2
0  6  7  0
1  8  2  5
2  3  1  4
step 9: 
   0  1  2
0  6  7  5
1  8  2  0
2  3  1  4
step 10: 
   0  1  2
0  6  7  5
1  8  2  4
2  3  1  0
step 11: 
   0  1  2
0  6  7  5
1  8  2  4
2  3  0  1
step 12: 
   0  1  2
0  6  7  5
1  8  0  4
2  3  2  1
step 13: 
   0  1  2
0  6  7  5
1  8  4  0
2  3  2  1
step 14: 
   0  1  2
0  6  7  0
1  8  4  5
2  3  2  1
step 15: 
   0  1  2
0  6  0  7
1  

True